# Anàlisi de sentiment

En aquesta secció es realitza un anàlisi de sentiment sobre els tuits, com exemple d'utilització de models de llenguatge pre-entrenats. En aquest cas, s'utilitza el model [DistilBERT](https://arxiv.org/abs/1910.01108) per a l'anàlisi de sentiment. Aquest model és una versió més lleugera del model [BERT](https://arxiv.org/abs/1810.04805), que és un model de llenguatge pre-entrenat que s'ha utilitzat amb molt bons resultats en diferents tasques de processament de llenguatge natural, com ara [anàlisi de sentiment](https://arxiv.org/abs/1905.05583), [classificació de text](https://arxiv.org/abs/1904.09077) o [extracció d'informació](https://arxiv.org/abs/1906.05237).

 En aquest cas, s'utilitza el model pre-entrenat per a l'anàlisi de sentiment en anglès.

## Càrrega del `dataset`

Utilitzarem la llibreria [datasets](https://huggingface.co/docs/datasets/) per a carregar el `dataset` de tuits. Aquesta llibreria permet carregar `datasets` de diferents fonts, com ara [Hugging Face Hub](https://huggingface.co/datasets), [Amazon AWS](https://docs.aws.amazon.com/es_es/marketplace/latest/userguide/datasets.html) o [Google Cloud](https://cloud.google.com/ai-platform/training/docs/datasets). En aquest cas, carregarem el `dataset` de tuits des de [Hugging Face Hub](https://huggingface.co/datasets/dair-ai/emotion).


In [51]:
# Instal·lem les lliberies que anem a utilitzar

%pip install --upgrade transformers datasets evaluate accelerate scikit-learn accuracy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
import datasets

# Carreguem el dataset
dataset = datasets.load_dataset('dair-ai/emotion')

# Mostrem les dades d'exemple

dataset['train'][0]

{'text': 'i didnt feel humiliated', 'label': 0}

Podem veure que cada registre del `dataset` conté el text del tuit i el sentiment associat. En aquest cas, el sentiment està codificat amb un enter entre 0 i 5, on 0 correspon a sadness, 1 a joy, 2 a love, 3 a anger, 4 a fear i 5 a surprise.

## Preparació del `dataset`

En aquest cas, el `dataset` ja està dividit en conjunts d'entrenament, test i validació. El següent pas és preparar el `dataset` per a l'entrenament del model. En aquest cas, el model que utilitzarem és el model [BERT](https://arxiv.org/abs/1810.04805). Aquest model requereix que el text estigui tokenitzat i que els tokens estiguin codificats amb els seus identificadors numèrics corresponents. Per a això, utilitzarem un tokenitzador de DistilBERT pre-entrenat.

In [53]:
# importem el tokenitzador de DistilBERT
from transformers import AutoTokenizer

# Carreguem el tokenitzador
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Mostrem un exemple de tokenització
tokenizer.tokenize('FC Barcelona is fucked this year')

['fc', 'barcelona', 'is', 'fucked', 'this', 'year']

In [54]:
# Definim una funció per pre-processar el text.
# Truncarem els textos per assegurar-nos que no superen el tamnay màxim d'entrada de DistilBert
# També ens assegurarem que totes les entrades tenen la mateixa longitud afegint padding

def tokenize(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

Per aplicar la tokenització, utilitzarem la funció `map` de `datasets`. Aquesta funció permet aplicar una funció a cada registre del `dataset`. En aquest cas, la funció que aplicarem és la funció `tokenize` que hem definit anteriorment. Utilitzarem també `batched=True` per a indicar que la funció s'aplicarà a tot el `dataset` en blocs.

In [55]:
dades_tokenitzades = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

## Avaluació

Per a avaluar el model, hem de carregar la métrica que utilitzarem per a l'avaluació. En aquest cas, utilitzarem la métrica `accuracy` del mòdul `evaluate` de HuggingFace.

També definirem una funció per a calcular les mètriques del model. Aquesta funció serà utilitzada per a avaluar el model després de cada epoch.

In [56]:
import evaluate

accuracy = evaluate.load('accuracy')

In [57]:
# Definim una funció per a calcular la precisió del model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

## Definició de les etiquetes

Abans d'entrenar el model hem de crear un diccionari que tradueixi els identificadors numèrics del sentiment a les seves etiquetes corresponents i a l'inrevés.

In [58]:
id_a_etiqueta = {
    0: "SADNESS",
    1: "JOY",
    2: "LOVE",
    3: "ANGER",
    4: "FEAR",
    5: "SUPRISE"
}

etiqueta_a_id = {
    "SADNESS": 0,
    "JOY": 1,
    "LOVE": 2,
    "ANGER": 3,
    "FEAR": 4,
    "SUPRISE": 5
}

## Fine tuning del model

El procés de fine tuning del model és el d'entrenar el model amb el nostre `dataset`. Aixó permet que el model s'adapte millor a les nostres dades i millori el seu rendiment.

Hem de definir la funció d'optimització, el tamany dels blocs i el nombre d'epochs.

In [59]:
BATCH_SIZE = 16
NUM_EPOCHS = 5

Ara ja podem carregar el model pre-entrenat i fer el fine tuning. Per entrenar ytilitzarem `AutoModelForSequenceClassification` i li agregarem les etiquetes que hem definit anteriorment.

In [60]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(etiqueta_a_id),
    id2label=id_a_etiqueta,
    label2id=etiqueta_a_id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Per utilitzar la funció d'avaluació que hem definit anteriorment, hem de definir un objecte `TrainingArguments` amb els paràmetres de l'entrenament. Podem incloure el nombre d'epochs, el tamany dels blocs, la mida del batch, la taxa d'aprenentatge, etc.

In [61]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

Ara ja podem entrenar el model, utilitzant el `trainer`.

In [62]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dades_tokenitzades['train'],
    eval_dataset=dades_tokenitzades['validation'],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## Inferència

Per a fer inferència amb el model, crearem un pipeline de HuggingFace. Aquest pipeline utilitzarà el model i el tokenitzador que hem importat anteriorment.

A continuació, utilitzarem el pipeline per a fer inferència amb un text d'exemple.

In [24]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
print(classifier("FC Barcelona is fucked this year"))
print(classifier("I'm not going to watch soccer again"))
print(classifier("Real Madrid is not going to treble this year. I'm relieved!"))

[{'label': 'ANGER', 'score': 0.9984560012817383}]
[{'label': 'SADNESS', 'score': 0.6338914632797241}]
[{'label': 'JOY', 'score': 0.9984081387519836}]
